In [1]:
import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('..')

In [2]:
from omegaconf import OmegaConf
import torch
import segmentation_models_pytorch as smp
from torch import nn
from pytorch_lightning.logging import TensorBoardLogger
from dl.lightning_model import *

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192

/usr/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning:

can't resolve package from __spec__ or __package__, falling back on __name__ and __path__

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192

/usr/l

In [3]:
model = smp.FPN(encoder_name="resnext50_32x4d", classes=2, decoder_merge_policy='cat')
# class_weight = dataset["train"].numpixels[1,0:2]
# class_weight = torch.from_numpy(1-class_weight/class_weight.sum()).type('torch.FloatTensor')
criterion = nn.CrossEntropyLoss(
    ignore_index = -100,
    reduction = 'none'
)

In [4]:
conf = OmegaConf.create(
    {
        'dirs': {
            'test_dir': "/home/leos/IMAGE_ANALYSIS/HDF5_data/databases/Kumar/patch_224/test_Kumar.pytable",
            'train_dir': "/home/leos/IMAGE_ANALYSIS/HDF5_data/databases/Kumar/patch_224/train_Kumar.pytable"
        },
        
        'dataloader_args': {
            'batch_size':6,
            'n_classes':2
        },
        
        'optimizer_args':{
            'lr':0.001,
            'encoder_lr':0.0005,
            'weight_decay':0.0003,
            'encoder_weight_decay':0.00003
        },
        
        'scheduler_args': {
            'factor':0.25,
            'patience':2
        },
    
        'loss_args' : {
            'edge_weight' : 1.1,
        }

    }
)
print(conf.pretty())

dirs:
  test_dir: /home/leos/IMAGE_ANALYSIS/HDF5_data/databases/Kumar/patch_224/test_Kumar.pytable
  train_dir: /home/leos/IMAGE_ANALYSIS/HDF5_data/databases/Kumar/patch_224/train_Kumar.pytable
dataloader_args:
  batch_size: 6
  n_classes: 2
optimizer_args:
  lr: 0.001
  encoder_lr: 0.0005
  weight_decay: 0.0003
  encoder_weight_decay: 3.0e-05
scheduler_args:
  factor: 0.25
  patience: 2
loss_args:
  edge_weight: 1.1



In [5]:
# config = {
#     'test_dir': "/home/leos/IMAGE_ANALYSIS/HDF5_data/databases/Kumar/patch_224/test_Kumar.pytable",
#     'train_dir': "/home/leos/IMAGE_ANALYSIS/HDF5_data/databases/Kumar/patch_224/train_Kumar.pytable", 
#     'batch_size': 6,
#     'lr': 0.001,
#     'encoder_lr': 0.0005,
#     'edge_weight':torch.tensor(1.1),
#     'model': model,
#     'criterion':criterion,
# }
logdir = "/home/leos/IMAGE_ANALYSIS/trained_dl_models/smp/Kumar/V1/patch_224/test_lightning/lightning_logs/"
model_dir = '/home/leos/IMAGE_ANALYSIS/trained_dl_models/smp/Kumar/V1/patch_224/test_lightning/'

In [6]:
%load_ext tensorboard
%tensorboard --logdir {logdir}

Reusing TensorBoard on port 6006 (pid 16892), started 1:02:53 ago. (Use '!kill 16892' to kill it.)

In [7]:
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    filepath = model_dir,
    save_top_k = 1,
    save_last = True,
    verbose = True, 
    monitor = 'val_loss',
    mode = 'min',
    prefix = ''
)

logger = TensorBoardLogger(save_dir=logdir, name="FPN")

/home/leos/.local/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning:

Checkpoint directory /home/leos/IMAGE_ANALYSIS/trained_dl_models/smp/Kumar/V1/patch_224/test_lightning/ exists and is not empty with save_top_k != 0.All files in this directory will be deleted when a checkpoint is saved!



In [8]:
lightning_model = SegModel(model=model, criterion = criterion, hparams = conf)
trainer = pl.Trainer(
    default_root_dir=model_dir,
    max_epochs=30, 
    gpus=1, 
    checkpoint_callback=checkpoint_callback, 
    logger=logger
)
trainer.fit(lightning_model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]
/usr/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning:

can't resolve package from __spec__ or __package__, falling back on __name__ and __path__


  | Name      | Type             | Params
-----------------------------------------------
0 | model     | FPN              | 25 M  
1 | criterion | CrossEntropyLoss | 0     



Epoch 00000: val_loss reached 0.24877 (best 0.24877), saving model to /home/leos/IMAGE_ANALYSIS/trained_dl_models/smp/Kumar/V1/patch_224/test_lightning/epoch=0.ckpt as top 1



Epoch 00001: val_loss reached 0.20884 (best 0.20884), saving model to /home/leos/IMAGE_ANALYSIS/trained_dl_models/smp/Kumar/V1/patch_224/test_lightning/epoch=1.ckpt as top 1



Epoch 00002: val_loss reached 0.20376 (best 0.20376), saving model to /home/leos/IMAGE_ANALYSIS/trained_dl_models/smp/Kumar/V1/patch_224/test_lightning/epoch=2.ckpt as top 1



Epoch 00003: val_loss reached 0.18919 (best 0.18919), saving model to /home/leos/IMAGE_ANALYSIS/trained_dl_models/smp/Kumar/V1/patch_224/test_lightning/epoch=3.ckpt as top 1


...


Epoch 00027: val_loss  was not in top 1



Epoch 00028: val_loss  was not in top 1



Epoch 00029: val_loss  was not in top 1


Epoch    30: reducing learning rate of group 0 to 4.8828e-07.
Epoch    30: reducing learning rate of group 1 to 4.8828e-07.
Epoch    30: reducing learning rate of group 2 to 4.8828e-07.
Epoch    30: reducing learning rate of group 3 to 4.8828e-07.
Epoch    30: reducing learning rate of group 4 to 4.8828e-07.
Epoch    30: reducing learning rate of group 5 to 4.8828e-07.
Epoch    30: reducing learning rate of group 6 to 4.8828e-07.
Epoch    30: reducing learning rate of group 7 to 4.8828e-07.
Epoch    30: reducing learning rate of group 8 to 4.8828e-07.
Epoch    30: reducing learning rate of group 9 to 4.8828e-07.
Epoch    30: reducing learning rate of group 10 to 4.8828e-07.
Epoch    30: reducing learning rate of group 11 to 4.8828e-07.
Epoch    30: reducing learning rate of group 12 to 4.8828e-07.
Epoch    30: reducing learning rate of group 13 to 4.8828e-07.
Epoch    30: reducing learning rate of group 14 to 4.8828e-07.
Epoch    30: reducing learning rate of group 15 to 4.8828e-07.
Ep

1